# Scipy 2021 Project - Sympy - Telegram - Bot - Magnus Müller and Felix Hammer

First of all the required libraries:

In [445]:
!pip install pyTelegramBotAPI
!pip install sympy
!pip install numpy
!pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


You can also find them in our requiremts.txt

## First let's get a grib on pyTelegramBotAPI

Here is the documentation: https://pypi.org/project/pyTelegramBotAPI/
But you don't really need it, because it is very simple:

In [453]:
import telebot # You first have to import the library (it goes by another name)

# Then you have to specify the API Key of your bot here as string: (You can create a bot with the telegram bot father and get an API Key)
API_KEY = "1913464534:AAGjZ9gC5KQmRiAjd-YWCAjhkKYGIpU6SBM"
bot = telebot.TeleBot(API_KEY) # and create a telebot with the api

# After that we just have to specify message handlers, which check wether a reply function is fitting:
@bot.message_handler(commands=['hello']) # commands accepts telegram comands (in this case "/hello")
def hello(message): # and then you just have to state the awnser function directly under the handler
    bot.send_message(message.chat.id, "Hello!") # in this case we simply send back an "Hello" to the person chatting with us
    
    
# You can also you as a handler boolean functions like this: 

def a_boolean_function(message):
    return message.text=='How are you?'

@bot.message_handler(func=a_boolean_function)
def reply(message): # and then you just have to state the awnser function directly under the handler
    bot.send_message(message.chat.id, "I am a bot, I don't have feelings")
    
# To run the bot you will then only need this command

bot.polling()  # use none_stop=True as argument to keep running after an error occured


This was basically everything we used, besides a simple send_photo command

It this important to note that the handlers are checked in the order they are defined by

## Then let's have a look at SymPy

SymPy is a great library for math 
First let's import it:

In [451]:
from sympy import *

x, y, z = symbols('x y z') # here we specify the variables, we are using

# Simpy offers nice mathematical tool like these:

function = parse_expr('exp(x**2)') # Define a function with an string
print(function)
print(function.diff()) # Derivate the Function
print(integrate(function)) # Integrate it 

exp(x**2)
2*x*exp(x**2)
sqrt(pi)*erfi(x)/2


These 3 functions are mainly the only functions, we are using

# Project’s goal, motivation and structure
## Goal
The goal of this project it to have a create a Telegram Bot which acts as a simple "Math-Helper"

So when you are sitting in front of you math-tasks and you want to quickly derivate a function you can just simple open telegram instead of googling for the classical "derivation- calculator" sites 

some
<span style="color:red">!!! Insert example pictures here
And write about format here text</span>.

## Motivation

The Motivation was basically that we like math and python and creating this telegram-bot seemed like an awesome way to combine those two. Furthermore, a telegram bot seemed pretty cool and we thought that getting some knowledge on how to create one might be handy later on.

## Structure 

This is the structure of our Bot:

<img src="img/screen.jpg">

It looks quite complex at first but, it is not that hard to understand.

So what we end up running is the main.py. The bot will then start polling and waiting for incoming messages.
If a message is received, the bot will check wether the message maps to a message-hanlder in the order the handlers are defined by.

The five big importand handlers are the ones for plotting, derivating, integrating, saving a function or computing operations over the saved function. They all call a boolean function from the boolean_funtions.py, with the exception of bool_commands. 

The boolean function then respectivly call a special parsing function from parsing_functions.py.


### parse_function
Since all functions are build similary, we will explain the structure based on the derivation progess, from the bottom up:


In [460]:
from sympy import *
from parsing_functions import parse_function
x, y, z = symbols('x y z')

# The most fundamental building plot is this parse_function:
parse_function('x2')

2*x

It allows us to transform a string of a mathematical function written in a correct way to be transformed to an sympy expression. 
This treats things differently as the parse_expr function from sympy:

In [458]:
parse_expr('x2')

x2

Here some more examples:

In [461]:
parse_function('sin(exp(cos(x**2)))')

sin(exp(cos(x**2)))

In [463]:
parse_function('notafuntion%') # returns a failure message if you can't parse the function

'Function not corretly formated'

In [464]:
parse_function('butthisisafunction')

butthisisafunction

### parse_derivation
This parse_function is now being used by the parse_derivation function:


In [475]:
def parse_derivation(string):
    """
    Input: string
    Output: sympy function or string if parsing failed or False if we the string does'nt start with 'derivate '
    """
    if string[0:9]=='derivate ':
        first = string.find("'")
        second = string.find("'",first+1)
        if first!= -1 and second !=-1: # if function correctly instantiated 
            pos_var=string.find('var')
            if pos_var!=-1: # if variable is specified
                if pos_var+4 < len(string) and string[pos_var+4] in ['x','y','z']: # if variable is corectly specified
                    erg = parse_function(string[first+1:second])
                    if type(erg)==str:
                        return erg
                    else:
                        return [erg,string[pos_var+4]] # return functionm variabke as list
                else:
                    return "No or wrong Variable given" # return failure message
            else:
                erg = parse_function(string[first+1:second])
                if type(erg)==str:
                    return erg
                else:
                    return [erg] # return function as list
        else:
            return 'Function not correctly instantiated'# return failure message
    return False


The function will be called with the text of the incomming message as argument.
If the text doesn't start with 'derivate ', the function will just return false:

In [476]:
parse_derivation('deriva')

False

If we get an input that is formated correctly we will get the parsed function possibly a variable in form of a list:

In [477]:
parse_derivation("derivate 'exp(sin(x**2))'")

[exp(sin(x**2))]

In [478]:
parse_derivation("derivate 'exp(sin(x**2))' var x")

[exp(sin(x**2)), 'x']

And if we detect some mistake, be it a wrong variable, or function that was instatntiated incorectly we get error messages as strings back:

In [479]:
parse_derivation("derivate 'missing_abostroph")

'Function not correctly instantiated'

In [480]:
parse_derivation("derivate ")

'Function not correctly instantiated'

In [481]:
parse_derivation("derivate '2x' var j")

'No or wrong Variable given'

In [482]:
parse_derivation("derivate '2x' var ")

'No or wrong Variable given'

In [483]:
parse_derivation("derivate '%&'")

'Function not corretly formated'

### bool_derivation
Then we will come to bool_derivation. This is the boolean function for the message handler.

In [ ]:
def bool_derivation(message):
    """
    Input : telegram message
    Output: Bool
    This functions is a boolean functions for a telegram bot handler
    It is suppose to check wether one tried to specify a function for derivation (doesn't matter wether is is instantiated correclty or is a correct mathematical expression)
    """
    string = message.text
    erg=parse_derivation(string)
    if erg!= False: # when parse_derivation does not return False we know that atleast someone tried to plot a function
        pickle.dump(str([(type(erg)!=str),str(erg)]),open("temp/buffer.dat","wb")) # save the data for later because then we don't have to run every parse function again
        return True
    else:
        return False


When the parser returns False, we know that the message was did not have anything to do with derivating, there fore we just return false. 

If the parser does'nt return False,we know that atleast someone tried to plot a function and that return value is either a failure message as a string or the function as a sympy expr.

We then save the data via the pickle module in the buffer.dat file. [(type(erg)!=str),str(erg)] we chose this format, so that we can later know wether the return value was a failure message (then the first item of the list is False)

Saving the data allows us to later not the same computations again


### Awnsering function: derivate
If the bool_derivation function detects a try to derivate and therefore returns True the function underneath of the derivation-handler will be called, in this case derivate from the main.py file.

In [484]:
@bot.message_handler(func=bool_derivation)
def derivate(message):
    """This function handels the derivation (working ones and failures)"""
    text=str(message.text)
    gespeichertes_erg = getting_buffer() # get the saved data from the buffer
    if gespeichertes_erg[0]==False: # if the saved data is a failure message
        bot.send_message(message.chat.id,gespeichertes_erg[1]) # just reply with the failure message
    else: # derivate integrate the function
        erg = gespeichertes_erg[1]
        rückgabe=''
        if len(erg)==2: # if var was specified
            rückgabe= str(get_derivative(erg[0],var=erg[1]))
        elif len(erg)==1: # if var was not specified
            rückgabe= str(get_derivative(erg[0]))
        saving_last_function(rückgabe,str(message.chat.id)) # save the result as last function 
        bot.send_message(message.chat.id, rückgabe.replace('**','^'))  # reply the function

First we get the data via the getting_buffer() function. If the data or gespeichertes_erg was a failure message, we just reply with the failure message.

If the data was a function we then call get_derivative from the math_part.py file on it, and reply the awnser as aditionally save the awnser as the last replied awnser via the saving_last_function.

This structure: handler -> boolean function -> parsing function -> saving data -> awnsering function getting data -> use special fucntion like plot or get_derivative -> send reply message 

is pretty much identical to the integrate, plotting or setting-function reply structures.

### The temp/ folder
In this folder we save the last correct function that the bot repplied to a messager, and the current function of the messager as will as the pictures of the plot